# Advanced RAG

정보 검색의 품질을 더욱 향상시키기 위해 사용되는 다양한 Advanced RAG 기술 중 4가지 기술에 대해서 설명한다.

- 검색된 문서의 순서를 변경하는 => 리랭커(Re-ranker)
- 가상의 문서를 생성하는 => HyDE(Hypothentical Document Expansion)
- 더 많은 정보를 포함하도록 쿼리를 변경하는 => 쿼리 확장
- 다양한 관점을 반영하도록 다중으로 쿼리를 생성하는 => 멀티 쿼리

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### (1) 리랭커(Re-ranker)

문서의 순위를 조절하는 전통적인 정보 검색(Information Retrieval, IR) 방법으로는
- TF-IDF (Term Frequency-Invert Document Frequency)
- BM25 (Best Matching 25)

두 방법 모두 특정 단어가 얼마나 중요한지를 평가하는 방식으로, 문서 검색의 초기 단계에서 많이 사용하는 대표적인 통계 기반 모델

##### TF-IDF

TF 는 특정 단어가 한 문서 내에서 얼마나 자주 등장하는지를 나타냄 -> 자주 등장할수록 중요한 단어로 간주됨  
IDF 는 특정 단어가 전체 문서에서 얼마나 드물게 등장하는지를 나타냄 -> 이는 흔히 등장하는 단어의 중요도를 줄여주는 역할

예시)  
전체 단어의 수: 100  
해당 문서에서 등장한 횟수: 3  
=> TF = 3/100

전체 문서의 개수: 4  
해당 단어가 등장한 문서의 개수: 2  
=> IDF = log(4/2)

TF-IDF = TF * IDF = (3/100) * log(4/2)

##### BM25

TF-IDF를 개선한 정보 검색 모델

문서와 쿼리 간의 관련성을 평가하는데 있어 보다 세밀한 조정을 가능하게 함

장점)  
문서의 길이에 따라 가중치를 조정하는 기능이 있어 짧은 문서와 긴 문서 간의 공정한 비교가 가능하다는 것

리랭커 모델은 크게 Cross-encoder 방식과 Bi-encoder 방식

- Cross-encoder 방식
  - 쿼리와 문서를 한 쌍으로 결합하여 입력으로 넣고, 두 텍스트가 얼마나 관련이 있는지에 대한 점수를 직접 계산하여 각 문서의 최종 순위를 재조정하는 방식
- Bi-encoder 방식
  - 쿼리와 문서를 독립적으로 처리하여 임베딩을 계산하는 방식
  - 문서에 대한 임베딩을 오프라인에서 미리 계산할 수 있다는 장점이 있음 -> 이는 대규모 검색 시스템에서 매우 중요한 효율성을 제공함
  - 쿼리와 문서를 독립적으로 임베딩하므로, 두 입력 간의 복잡한 상호작용을 잘 반영하지 못한다는 한계가 있음

In [12]:
!pip install torch transformers pymupdf sentence-transformers hf_xet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 63.9 MB/s eta 0:00:0000:0100:01


In [5]:
# FAISS 를 통해서 최근접 이웃을 먼저 찾고, 이를 Cross-encoder 기반의 리랭커 모델을 사용하여 직접적인 스코어를 산출하여 문서를 재정렬 해보자.

# PDF 파일에서 -> 텍스트 추출 -> HuggingFaceEmbeddings 를 사용하여 임베딩 -> FAISS 벡터 스토어에 저장

import torch
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import AutoModelForSequenceClassification, AutoTokenizer


/home/haiqv/conda/envs/agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# PDF 로드
loader = PyMuPDFLoader("BBS_202402151054353090.pdf")
documents = loader.load()

In [13]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Document 객체에서 텍스트 내용 추출
text_contents = [doc.page_content for doc in texts]

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 벡터 스토어 설정
vectorstore = FAISS.from_texts(text_contents, embeddings)

In [14]:
# BGE Reranker 모델 설정
model_name = "BAAI/bge-reranker-v2-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [15]:
# 리랭킹을 수행하고 기본 검색 결과와 비교해 보자

def rerank_documents(query, docs, top_k=5):
    pairs = [[query, doc.page_content] for doc in docs]
    with torch.no_grad():
        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors="pt", max_length=512)
        scores = model(**inputs).logits.squeeze(-1)
    ranked_indices = scores.argsort(descending=True)
    reranked_docs = [(docs[i], scores[i].item()) for i in ranked_indices[:top_k]]
    return reranked_docs

In [16]:
# 리트리버 설정
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

# 쿼리 실행
query = "국내 고령화 전망에 대해 알려주세요"

# 기본 검색 결과 출력
print("## 기본 검색 결과")
base_docs = base_retriever.invoke(query)
for i, doc in enumerate(base_docs[:5]):
    print(f"문서 {i+1}:")
    print(doc.page_content[:100] + "...\n")

# 리랭킹 결과 출력
print("\n## 리랭킹 결과")
reranked_docs = rerank_documents(query, base_docs)
for i, (doc, score) in enumerate(reranked_docs):
    print(f"문서 {i+1} (점수: {score:.4f}):")
    print(doc.page_content[:100] + "...\n")

## 기본 검색 결과
문서 1:
공공기관의 돌봄서비스 제공에 활용할 수 있는 기술개발이 필요
 신체 및 정서 변화를 겪는 고령층을 위한 다양한 기술과 서비스는 글로벌 사회가 직면한 
초고령 사회의 문제점을 해결...

문서 2:
고령화시대 해결을 위한 기술개발
- 19 -
◎ 돌봄 분야
 (한국, 스마트요양원) 2020년부터 부산에서 요양원을 디지털 전환하는 ‘스마트 요양원 사업’을 
진행하여 현장에 적...

문서 3:
TIPA 이슈 리포트
Vol. 6
- 14 -
Ⅲ. 고령화 기술·제품 동향
◎ 신체활동/이동 보조 분야
 (한국, 스마트 지팡기) 말하는 스마트 지팡기 ‘톡톡스틱’은 기존 지팡이...

문서 4:
고령화시대 해결을 위한 기술개발
- 17 -
◎ 정서/인지 케어 분야
 (한국, 인지치료 로봇) ICT 인지중재 프로그램이 탑재된 앵무새 로봇 ‘피오’
▪ 경도인지장애 및 초기치...

문서 5:
정기적으로 전달하고 응답내용을 분석하여 위험에 처하거나 특별한 주의가 필요하다고 인식되면 응급 
알람 메시지를 가족이나 의사에게 전달
▪ 시니어의 건강상태가 좋지않아 즉각적인 도움...


## 리랭킹 결과
문서 1 (점수: 3.0630):
고령화시대 해결을 위한 기술개발
- 3 -
Ⅰ. 고령화시대의 도래
1
국내외 고령화 현황 및 전망
 고령화(高齡化)란 평균 수명의 증가에 따라 총 인구 중 차지하는 노인의 인구비...

문서 2 (점수: -1.5022):
고령화시대 해결을 위한 기술개발
- 7 -
 정부는 고령친화 우수사업자·고령친화 우수제품 지정제도 도입으로 안심하고 고령친화사업의 
용품을 이용할 수 있도록 하고 있으나 고령친화...

문서 3 (점수: -2.1617):
고령화시대 해결을 위한 기술개발
- 21 -
Ⅴ. 결론 및 시사점
 현재 고령친화산업은 사물인터넷(IoT), 모바일과 인공지능의 발달과 비대면 서비스의 확대로 
국내외 관심이 더...

문서 4 (점수: -2.7052):
TIPA 이슈 리포트
Vol. 6
- 20 -
Ⅳ.

### (2) HyDE

Hypothetical Document Embeddings

쿼리에 대한 가상 문서(Hypothetical Document)를 생성하고, 이를 기반으로 검색을 수행하는 방식  
벡터 스토어로의 검색은 하지 않는다.

기존의 BM25 나 TF-IDF 같은 전통적인 정보 검색 기법과 함께 사용되며, 특히 신경망 기반 리랭커와의 조합을 통해 높은 성능을 발휘한다.

In [2]:
# HyDE로 가상의 문서를 생성하고, 이를 바탕으로 검색하는 실습

import torch
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HypotheticalDocumentEmbedder
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.prompts import PromptTemplate

/home/haiqv/conda/envs/agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# PDF 로드
loader = PyMuPDFLoader("BBS_202402151054353090.pdf")
documents = loader.load()

In [4]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
text_contents = [doc.page_content for doc in texts]

In [5]:
# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 벡터 스토어 설정
vectorstore = FAISS.from_texts(text_contents, embeddings)

/tmp/ipykernel_553/2212684356.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


In [6]:
# Qwen/Qwen2.5-1.5B-Instruct 모델 설정
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,  # 생성할 최대 새 토큰 수
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        device=0 if torch.cuda.is_available() else -1
    )
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda:0
/tmp/ipykernel_553/3568116609.py:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(


In [7]:
# 사용자 정의 프롬프트 템플릿 생성
custom_prompt = PromptTemplate(
    input_variables=["question"],
    template="다음 질문에 대한 가상의 문서를 생성해주세요: {question}\n\n문서:"
)

# HyDE 설정
hyde = HypotheticalDocumentEmbedder.from_llm(
    llm=llm,
    base_embeddings=embeddings,
    custom_prompt=custom_prompt
)

# 쿼리 실행
query = "국내 고령화 전망에 대해 알려주세요"
hyde_embedding = hyde.embed_query(query)

# vectorstore를 사용하여 검색
results = vectorstore.similarity_search_by_vector(hyde_embedding)

# hyde.embed_query() 호출 전후에 중간 결과 출력
print("생성된 가상 문서:")
print(llm(custom_prompt.format(question=query)))
print("생성된 임베딩:", hyde_embedding[:10])  # 임베딩의 처음 10개 값만 출력

# 결과 출력
for i, doc in enumerate(results[:5]):  # 상위 5개 결과만 출력
    print(f"관련 문서 {i+1}:")
    print(doc.page_content[:100] + "...\n")

Using a deprecated class. Please use `from langchain.chains import HypotheticalDocumentEmbedder` instead


생성된 가상 문서:


/tmp/ipykernel_553/485704254.py:23: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm(custom_prompt.format(question=query)))


다음 질문에 대한 가상의 문서를 생성해주세요: 국내 고령화 전망에 대해 알려주세요

문서: 국내 고령화 전망

고령화는 세계적으로 인기 있는 이슈 중 하나로, 한국也不例外입니다. 한국은 2018년부터 고령화 사회로 진입하였으며, 이에 따른 변화와 위협이 대두되고 있습니다.

한국의 고령화 현상을 이해하기 위해서는 주요 특징과 미래 전망을 살펴보는 것이 중요합니다. 

첫째, 고령화 시대의 가장 큰 문제점은 노인 심身 건강 문제입니다. 노인이 점점 늙어지면서 다양한 건강 문제가 발생하고 있어 이를 해결하기 위한 노력이 필요합니다. 

둘째, 고령화는 경제에도 영향을 미칩니다. 고령화 사회에서는 더 많은 노인들이 일자리를 찾는데 어려움을 겪게 됩니다. 따라서 정책의 한계로, 고령화가 경제적 안정성을 약화시키는 요인으로 작용할 수 있습니다.

셋째, 고령화는 교육 분야에서도 중요한 변화를 가져올 것입니다. 교육의 형태가 변화하면서 더 많은 학생들이 고령의 부모님을 돌볼 수 있도록
생성된 임베딩: [np.float64(0.04337908327579498), np.float64(0.25073957443237305), np.float64(0.07979312539100647), np.float64(-0.10748119652271271), np.float64(0.17996497452259064), np.float64(0.07843017578125), np.float64(-0.14852800965309143), np.float64(0.11208377033472061), np.float64(0.02158929780125618), np.float64(0.23853909969329834)]
관련 문서 1:
고령화시대 해결을 위한 기술개발
- 3 -
Ⅰ. 고령화시대의 도래
1
국내외 고령화 현황 및 전망
 고령화(高齡化)란 평균 수명의 증가에 따라 총 인구 중 차지하는 노인의 인구비...

관련 문서 2:
고령화시대 해결을 위한 기술개발
- 5 -
2
고령친화 기술의 필요성
 국

### (3) 쿼리 확장 (Query Expansion)

쿼리를 보다 구체적이고 풍부하게 만드는 기술

쿼리 확장은 크게 2가지 방식
- 통계적 접근: 자주 함께 등장하거나 맥락적으로 유사한 단어를 추가하여 검색 범위를 확장
- 신경망 접근: 딥러닝을 통해 맥락적으로 관련된 단어를 추가하여 의미적 이해를 향상시킨다


In [8]:
import torch
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [9]:
# PDF 로드 및 텍스트 분할
loader = PyMuPDFLoader("BBS_202402151054353090.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
text_contents = [doc.page_content for doc in texts]

In [10]:
# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 벡터 스토어 설정
vectorstore = FAISS.from_texts(text_contents, embeddings)

# Qwen/Qwen2.5-1.5B-Instruct 모델 설정
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.3,
        top_p=0.85,
        repetition_penalty=1.2,
        device=0 if torch.cuda.is_available() else -1
    )
)

Device set to use cuda:0


In [12]:
# Query expansion 함수
def expand_query(original_query):
    expansion_prompt = f"다음 질문을 확장하여 관련된 다양한 키워드와 문구를 간결하게 한국어로 생성해주세요. 설명이나 문장 형태의 답변은 하지 마세요. 원래 질문: '{original_query}'"
    expanded_query = llm(expansion_prompt).split(original_query)[-1]
    return expanded_query

# 원래 쿼리
original_query = "국내 고령화 전망에 대해 알려주세요"

# 쿼리 확장
expanded_query = expand_query(original_query)
print(f"확장된 쿼리: {expanded_query}")

확장된 쿼리: ' 

1) 국내 고령화는 어떤 특징이 있는가?
2) 이에 따른 사회적 문제들은 무엇인가요? 
3) 어떻게 대응해야 할까요?

각 항목마다 추가적인 정보나 분석, 예시 등을 포함한 문장을 만들어보세요.
4) 국내 고령화 현상과 그에 따른 변화를 이해하기 위해 필요한 데이터 및 통계 자료는 어디에서 찾을 수 있을까? 5) 고령사회에 적합한 교육 시스템 개선 방안들을 제안할 수 있나요? 6) 경제적으로 안정적이면서도 건강하고 활기찬 노년기를 위한 정책들이 필요하다고 생각합니다.

7) 이러한 주제에 대한 연구 또는 논문 작성에 있어 중요한 참고자료들 중 하나는 무엇일까요? 8) 고령화에 따라 생활 패턴이 바뀌며, 이를 반영하는 새로운 서비스나 제품들의 발전 가능성은 어느 정도라고 볼 수 있을까요? 9) 고령화 사회에서는 인공 지능(AI), 빅데이터 등 기술 도입으로 인해 일자리 창출 효과가 크다고 주장하는데, 실제로 이런 영향이 미치는지는 어떠한 방법으로 파악할 수 있을까요? 10) 현재까지 이루어진 고령화 대응策 실험 결과에 대해서 조사하면 좋을 것 같습니다. 특히, 어떤 지역이나 단체에서 실증 프로그램을 진행했는지를 찾아볼 수 있습니다. 그리고 그 과정에서 발견되는 성공 사례들과 실패 사례들도 함께 살펴보고 싶습니다. 11) 고령화에 따른 소비 트렌드 변화에 대해 알아봅니다. 예를 들어, 젊은 세대보다 더 많은 돈을 쓰는 것이 일반적이라고 한다면, 이것이 어떤 의미인지 해설하시거나, 혹은 다른 사람들이 보기에 가장 큰 변화였던 부분은 무엇인지도 말해주시겠어요? 12) 고령화에 따른 교통수단 사용 변화에 대해 이야기하겠습니다. 저에게서 주변 사람들의 행동 변화를 알릴 수 있도록 하는 것은 중요하지만, 또한 이 변화가 사회 전체에 미치는 영향력도 언급해주실 수 있으신가요? 13) 고령화 사회에서 공감 능력을 갖춘 리더십 역량이 요구되는데, 이는 개인뿐만 아니라 조직 내에서도 적용될 수 있다고 합니다. 따라서, 이에 대한 강조점은 무엇이며, 실제 삶 

In [13]:
# 확장된 쿼리로 검색 수행
search_results = vectorstore.similarity_search(expanded_query, k=5)

# 결과 출력
for i, doc in enumerate(search_results):
    print(f"관련 문서 {i+1}:")
    print(doc.page_content[:100] + "...\n")

관련 문서 1:
고령화시대 해결을 위한 기술개발
- 9 -
▪ 2017년 건강노화 제품 및 서비스에 대한 R&D, 파일럿 테스트 프로젝트, 예비 상업화 프로젝트 외 
50건의 프로젝트를 진행 중임...

관련 문서 2:
고령화시대 해결을 위한 기술개발
- 7 -
 정부는 고령친화 우수사업자·고령친화 우수제품 지정제도 도입으로 안심하고 고령친화사업의 
용품을 이용할 수 있도록 하고 있으나 고령친화...

관련 문서 3:
증가 대응방안으로는 고령층 新수요에 대응하는 고령친화 산업 육성, 고령자 맞춤형 주거지원, 
고령친화도시 조성 등 포함
▪ 고령자가 시설이 아닌 가정과 지역사회에서 계속 거주하면서...

관련 문서 4:
TIPA 이슈 리포트
Vol. 6
- 6 -
Ⅱ. 고령화 정책 및 산업 동향
1
고령화 관련 정책동향
◎ 국내 정책동향
  2020년 보건복지부에서 제4차 저출산･고령사회기본계획...

관련 문서 5:
고령화시대 해결을 위한 기술개발
- 5 -
2
고령친화 기술의 필요성
 국내 생산가능인구 감소 및 노년부양비 증가 등 고령화 시대 도래에 따른 문제점을 극복하기 
위한 해결책이 ...



In [ ]:
import gc

del model
del llm

# 2. 가비지 컬렉션 수동 실행
gc.collect()

# 3. GPU 메모리 비우기
if torch.cuda.is_available():
    torch.cuda.empty_cache()

### (4) 멀티 쿼리

사용자가 입력한 단일 쿼리를 여러 개의 하위 쿼리로 나누거나, 여러 가지 관점에서의 쿼리를 생성하여 검색을 수행

In [14]:
import torch
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import logging
from langchain_core.prompts import PromptTemplate
from typing import List
from langchain_core.output_parsers import BaseOutputParser

In [15]:
# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('langchain.retrievers.multi_query')
logger.setLevel(logging.INFO)

In [21]:
# PDF 로드 및 텍스트 분할
loader = PyMuPDFLoader("BBS_202402151054353090.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
text_contents = [doc.page_content for doc in texts]

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 벡터 스토어 설정
vectorstore = FAISS.from_texts(text_contents, embeddings)

# LLM 설정 (Qwen/Qwen2.5-1.5B-Instruct 사용)
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        device=0 if torch.cuda.is_available() else -1
    )
)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Device set to use cuda:0


In [22]:
# 사용자 정의 프롬프트 템플릿 생성
custom_prompt = PromptTemplate(
    input_variables=["question"],
    template="""당신은 AI 언어 모델 어시스턴트입니다. 사용자가 제공한 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 질문을 3가지 다른 버전으로 생성하는 것이 당신의 임무입니다.
사용자의 질문을 다양한 관점에서 재구성하여 거리 기반 유사도 검색의 한계를 극복할 수 있도록 돕는 것이 목표입니다.
각 버전의 질문은 줄바꿈으로 구분하여 작성하세요.
한국어로 작성하세요. 원본 질문: {question}"""
)

# 출력 파서 정의
class LineListOutputParser(BaseOutputParser):
    def parse(self, text: str) -> List[str]:
        return text.strip().split("\n")

# LLM 체인 생성
output_parser = LineListOutputParser()
llm_chain = custom_prompt | llm | output_parser

# 다중 쿼리 리트리버 설정
retriever_from_llm = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(),
    llm_chain=llm_chain,
    parser_key="lines"
)
retriever_from_llm.verbose = True

# 쿼리 실행
query = "국내 고령화 전망에 대해 알려주세요"
results = retriever_from_llm.get_relevant_documents(query)

# 결과 출력
for i, doc in enumerate(results[:5]):  # 상위 5개 결과만 출력
    print(f"관련 문서 {i+1}:")
    print(doc.page_content[:100] + "...\n")

/tmp/ipykernel_553/2754249418.py:29: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever_from_llm.get_relevant_documents(query)
INFO:langchain.retrievers.multi_query:Generated queries: ['당신은 AI 언어 모델 어시스턴트입니다. 사용자가 제공한 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 질문을 3가지 다른 버전으로 생성하는 것이 당신의 임무입니다.', '사용자의 질문을 다양한 관점에서 재구성하여 거리 기반 유사도 검색의 한계를 극복할 수 있도록 돕는 것이 목표입니다.', '각 버전의 질문은 줄바꿈으로 구분하여 작성하세요.', '한국어로 작성하세요. 원본 질문: 국내 고령화 전망에 대해 알려주세요. 버전1: 국내 고령화 현황과 예상 변화를 설명해주세요. 버전2: 국내에서 인구 고령화 현상을 어떻게 볼 수 있나요? 버전3: 한국 고령화 상황의 현재 상태와 미래发展趋势에 대한 정보를 제공해주세요.']


관련 문서 1:
TIPA 이슈 리포트
Vol. 6
- 14 -
Ⅲ. 고령화 기술·제품 동향
◎ 신체활동/이동 보조 분야
 (한국, 스마트 지팡기) 말하는 스마트 지팡기 ‘톡톡스틱’은 기존 지팡이...

관련 문서 2:
 (일본, DMM PALMI) 노인용 대화로봇으로써 분위기 등을 감지하여 상대의 말 예측 
가능하고, 데이터가 쌓이면 말투를 비롯한 말솜씨가 발전, 상대방의 과거 대화내용·취미 ...

관련 문서 3:
고령화시대 해결을 위한 기술개발
- 15 -
[ 보행 재활기구, 트리 ]
[ 배설지원 로봇 ]
* 출처: : Shinkachi-portal
* 출처: 일본 사이버다인, 배설지원 로...

관련 문서 4:
정기적으로 전달하고 응답내용을 분석하여 위험에 처하거나 특별한 주의가 필요하다고 인식되면 응급 
알람 메시지를 가족이나 의사에게 전달
▪ 시니어의 건강상태가 좋지않아 즉각적인 도움...

관련 문서 5:
▪ 돌봄 분야 그동안의 개발은 공급자 중심으로 이루어졌지만, 수많은 기술개발의 성공에도 사업화로 
이어진 사례는 미미하였으며, 주요 원인으로 수요자의 요구와 피드백을 직접적이고 상...

